In [128]:
import numpy as np
import pandas as pd

import tensorflow as tf
import matplotlib.pyplot as plt
import keras 

from keras.callbacks import ModelCheckpoint
from keras.layers import Input, Dense, Dropout
from keras.models import Model
from sklearn.model_selection import train_test_split

"""
    Created by Mohsen Naghipourfar on 3/26/18.
    Email : mn7697np@gmail.com
    Website: http://ce.sharif.edu/~naghipourfar
"""

'\n    Created by Mohsen Naghipourfar on 3/26/18.\n    Email : mn7697np@gmail.com\n    Website: http://ce.sharif.edu/~naghipourfar\n'

In [129]:
# Hyper-Parameters
LEARNING_RATE = 1e-3
DROP_OUT = 0.5
N_SAMPLES = 10787
N_FEATURES = 19671
N_DISEASES = 34
N_BATCHES = 256
N_EPOCHS = 50
N_BATCH_LEARN = 10
N_RANDOM_FEATURES = 200
neurons = {
    'in': 200,
    'l1': 1024,
    'l2': 512,
    'l3': 256,
    'l4': 128,
    'out': N_DISEASES
}


In [117]:
def modify_output(target):
    global N_DISEASES
    output_dict = {}
    a = target[0].value_counts()
    # print(a)
    a = pd.DataFrame(a)
    i = 0
    for row in a.itertuples():
        output_dict[row[0]] = i
        i += 1
    N_DISEASES = i
    new_output = [0 for __ in range(N_SAMPLES)]
#     new_output = [[0 for _ in range(N_DISEASES)] for __ in range(N_SAMPLES)]
    for idx, y in target.iterrows():
#         new_output[idx][output_dict[y[0]]] = 1
        new_output[idx] = output_dict[y[0]]
    return new_output

In [118]:
# Load Data
x_data = pd.read_csv("../Data/fpkm_normalized.csv", header=None)
# y_data = pd.read_csv("../Data/disease.csv", header=None)
# y_data = pd.DataFrame(modify_output(y_data))

In [119]:
y_data = pd.read_csv("../Data/disease.csv", header=None)
y_data = pd.DataFrame(modify_output(y_data))
y_data = pd.DataFrame(keras.utils.to_categorical(y_data, num_classes=N_DISEASES))

In [120]:
# Random Feature Selection
random_feature_indices = np.random.choice(19671, N_RANDOM_FEATURES, replace=False)
x_data_new = x_data[random_feature_indices]

# Train/Test Split
x_train, x_test, y_train, y_test = train_test_split(x_data_new, y_data, test_size=0.20)

In [44]:
# x_train.columns = range(x_train.shape[1])
# x_test.columns = range(x_test.shape[1])

False

In [130]:
# Design Model
input_layer = Input(shape=(neurons['in'],))

l1 = Dense(neurons['l1'], activation='relu')(input_layer)

l1_dropout = Dropout(DROP_OUT)(l1)

l2 = Dense(neurons['l2'], activation='relu')(l1_dropout)

l2_dropout = Dropout(DROP_OUT)(l2)

l3 = Dense(neurons['l3'], activation='relu')(l2_dropout)

l3_dropout = Dropout(DROP_OUT)(l3)

l4 = Dense(neurons['l4'], activation='relu')(l3_dropout)

l4_dropout = Dropout(DROP_OUT)(l4)

output_layer = Dense(neurons['out'], activation='softmax')(l4_dropout)

# Compile Model
network = Model(input_layer, output_layer)

network.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

network.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 200)               0         
_________________________________________________________________
dense_46 (Dense)             (None, 1024)              205824    
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_47 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_48 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
__________

/Users/Future/anaconda/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/Users/Future/anaconda/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/Users/Future/anaconda/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/Users/Future/anaconda/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspe

In [51]:
from keras.callbacks import Callback
class TestCallback(Callback):
    def __init__(self, test_data):
        self.test_data = test_data

    def on_epoch_end(self, epoch, logs={}):
        x, y = self.test_data
        loss, acc = self.model.evaluate(x, y, verbose=0)
        print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

In [131]:
# Train Model
save_path = '../Results/Keras/' + 'weights.{epoch:02d}-{val_acc:.4f}.hdf5'
checkpointer = ModelCheckpoint(filepath=save_path,
                               verbose=1,
                               monitor='val_acc',
                               save_best_only=True,
                               mode='auto',
                               period=10)

network.fit(x=x_train.as_matrix(),
            y=y_train.as_matrix(),
            epochs=N_EPOCHS,
            batch_size=N_BATCHES,
            shuffle=True,
            validation_data=(x_test.as_matrix(), y_test.as_matrix()),
            callbacks=[checkpointer])

# TODO: Use Callback function for our network!

/Users/Future/anaconda/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/Users/Future/anaconda/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/Users/Future/anaconda/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/Users/Future/anaconda/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspe

Train on 8629 samples, validate on 2158 samples
Epoch 1/50
8629/8629 [==============================] - 2s - loss: 3.3167 - acc: 0.0986 - val_loss: 2.8929 - val_acc: 0.1747
Epoch 2/50
8629/8629 [==============================] - 0s - loss: 2.6518 - acc: 0.2584 - val_loss: 1.9047 - val_acc: 0.5042
Epoch 3/50
8629/8629 [==============================] - 0s - loss: 1.9330 - acc: 0.4641 - val_loss: 1.2142 - val_acc: 0.6789
Epoch 4/50
8629/8629 [==============================] - 0s - loss: 1.4143 - acc: 0.5988 - val_loss: 0.8884 - val_acc: 0.7549
Epoch 5/50
8629/8629 [==============================] - 0s - loss: 1.1221 - acc: 0.6735 - val_loss: 0.7198 - val_acc: 0.7859
Epoch 6/50
8629/8629 [==============================] - 0s - loss: 0.9753 - acc: 0.7110 - val_loss: 0.6170 - val_acc: 0.8137
Epoch 7/50
8629/8629 [==============================] - 0s - loss: 0.8262 - acc: 0.7590 - val_loss: 0.5426 - val_acc: 0.8392
Epoch 8/50
8629/8629 [==============================] - 0s - loss: 0.7498 - a

In [112]:
# keras.utils.plot_model(network, to_file='model.png', show_shapes=False, show_layer_names=True, rankdir='TB')

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.

b"%!PS-Adobe-3.0\n%%Creator: graphviz version 2.40.1 (20161225.0304)\n%%Title: G\n%%Pages: (atend)\n%%BoundingBox: (atend)\n%%EndComments\nsave\n%%BeginProlog\n/DotDict 200 dict def\nDotDict begin\n\n/setupLatin1 {\nmark\n/EncodingVector 256 array def\n EncodingVector 0\n\nISOLatin1Encoding 0 255 getinterval putinterval\nEncodingVector 45 /hyphen put\n\n% Set up ISO Latin 1 character encoding\n/starnetISO {\n        dup dup findfont dup length dict begin\n        { 1 index /FID ne { def }{ pop pop } ifelse\n        } forall\n        /Encoding EncodingVector def\n        currentdict end definefont\n} def\n/Times-Roman starnetISO def\n/Times-Italic starnetISO def\n/Times-Bold starnetISO def\n/Times-BoldItalic starnetISO def\n/Helvetica starnetISO def\n/Helvetica-Oblique starnetISO def\n/Helvetica-Bold starnetISO def\n/Helvetica-BoldOblique starnetISO def\n/Courier starnetISO def\n/Courier-Oblique starnetISO def\n/Courier-Bold starnetISO def\n/Courier-BoldOblique starnetISO def\ncleartoma